# The `Signal` class

The `Signal` class is a container based on the `np.array` storing besides the samples as function of time also a sample frequency. The class has many usefull methods making the life of an acoustician a bit easier. 
Furthermore, because its based on the `np.array`, all operations you would normally perform on a `np.array` can be done on a `Signal` as well. 

In [2]:
from acoustic_toolbox import Signal
import numpy as np
%matplotlib inline
from IPython.display import Audio
# import seaborn as sns # seaborn for prettier graphs

## Creating an instance
A `Signal` takes two arguments, the first an array of samples, and the second a sample frequency.

In [3]:
s = Signal(np.random.randn(44100), fs=11025)

Alternatively, one of the alternative constructors can be used

In [4]:
s = Signal.from_wav('../data/recording.wav')

FileNotFoundError: [Errno 2] No such file or directory: '../data/recording.wav'

## Attributes

An instance of `Signal` stores besides the instantaneous values also the sample frequency

In [4]:
s.fs

11025

The `Signal` class can handle multiple signals. Currently we have only one channel.

In [5]:
s.channels

1

The amount of samples in the signal can be retrieved in several ways. We can use the standard ways, `len(s)`, or `s.shape` but also the method `s.samples`

In [6]:
s.samples

44100

##Play signal with IPython
The recording is of an aircraft flyover as you can hear.

In [9]:
Audio(data=s, rate=s.fs)

## Spectral analysis

Let's continue for now with analysing the recording.

We can start with looking at the spectrum. A narrowband power spectrum can be calculated using `s.power_spectrum()`

In [8]:
s.power_spectrum()

(array([0.00000e+00, 2.50000e-01, 5.00000e-01, ..., 5.51175e+03,
        5.51200e+03, 5.51225e+03]),
 array([2.64040404e-05, 2.01824487e-05, 3.79970684e-05, ...,
        3.08625276e-05, 1.83114522e-05, 1.51925622e-05]))

returning frequencies and powers. It's also possible to straightaway plot the power spectrum

In [9]:
fig = s.plot_power_spectrum()

There are other types of spectra available, like for example the phase spectrum. Another popular figure is the spectrogram

In [10]:
fig = s.spectrogram()

## Sound pressure level

The methods shown so far are quite common operations, not specific to acoustics alone. But, the `Signal` class has some more acoustics-specific methods as well. For example, we can calculate the sound pressure level as function of time using `s.levels()` or instead plot the values 

In [11]:
fig = s.plot_levels()

## Fractional octaves

The label is the channel number, using zero-based numbering as is common in Python. We can also calculate and plot 1/1-octaves with respectively `s.octaves()` and `s.plot_octaves()`.

In [12]:
fig = s.plot_octaves()

Calculating or plotting 1/3-octaves is also possible.

In [13]:
fig = s.plot_third_octaves()

## Frequency weighting

The `acoustics` package features frequency weighting in several ways. When having a signal like shown in this example, frequency weighting can be applied directly using the `weigh` method.


In [14]:
s.weigh('A')

This can be seen better when plotting in octaves.

In [15]:
fig = s.weigh('A').plot_octaves()

## Descriptors

The signal class also features several descriptors, like e.g. $L_{eq}$.
Let's calculate as an example the A-weighted equivalent level over the event, i.e. $L_{A,eq}$.

In [16]:
s.weigh('A').leq()

This value is a bit below the unweighted $L_{Z,eq}$.

In [17]:
s.leq()

We can verify the calculation by using a different method. Let's calculate 1/3-octave values first, and then add the weighting. The A-weighting is

In [18]:
from acoustic_toolbox.standards.iec_61672_1_2013 import WEIGHTING_VALUES
WEIGHTING_VALUES['A']

and the A-weighted $L_{A,eq}$ determined using this method is

In [19]:
from acoustic_toolbox.decibel import dbsum
dbsum( s.third_octaves()[1] + WEIGHTING_VALUES['A'] )

## Multichannel signal

The Signal container can store more then one channel


In [20]:
s2 = Signal([s,s,s], s.fs)

as we can see

In [21]:
s2.channels

Most methods still work when considering multichannel signals.

In [22]:
fig = s2.weigh('A').plot_third_octaves()

## Exporting to wav

Finally, a signal can be written to WAV format easily as well using `s.to_wav(filename)`. You might want to normalize your signal first though, so e.g. `s.normalize().to_wav(filename)`.